In [1]:
import pandas as pd

In [2]:
# !git clone https://github.com/wshuyi/demo-chinese-text-classification-lstm-keras.git

In [3]:
from pathlib import Path

In [4]:
mypath = Path(".")
# Path("demo-chinese-text-classification-lstm-keras")

In [5]:
df = pd.read_csv(mypath/'dianping.csv')

In [6]:
df.head()

,comment,sentiment
0,口味：不知道是我口高了，还是这家真不怎么样。 我感觉口味确实很一般很一般。上菜相当快，我敢说...,0
1,菜品丰富质量好，服务也不错！很喜欢！,1
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,0
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,1
4,先说我算是娜娜家风荷园开业就一直在这里吃 每次出去回来总想吃一回 有时觉得外面的西式简餐总是...,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
comment      2000 non-null object
sentiment    2000 non-null int64
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


In [8]:
# !pip install jieba

In [9]:
import jieba

In [10]:
df['text'] = df.comment.apply(lambda x: " ".join(jieba.cut(x)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LIJIAX~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.926 seconds.
Prefix dict has been built succesfully.


In [11]:
df.head()

,comment,sentiment,text
0,口味：不知道是我口高了，还是这家真不怎么样。 我感觉口味确实很一般很一般。上菜相当快，我敢说...,0,口味 ： 不 知道 是 我口 高 了 ， 还是 这家 真 不怎么样 。 我 感觉 口味 ...
1,菜品丰富质量好，服务也不错！很喜欢！,1,菜品 丰富 质量 好 ， 服务 也 不错 ！ 很 喜欢 ！
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,0,说真的 ， 不 晓得 有人 排队 的 理由 ， 香精 香精 香精 香精 ， 拜拜 ！
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,1,菜量 实惠 ， 上菜 还 算 比较 快 ， 疙瘩汤 喝出 了 秋日 的 暖意 ， 烧茄子 吃...
4,先说我算是娜娜家风荷园开业就一直在这里吃 每次出去回来总想吃一回 有时觉得外面的西式简餐总是...,1,先说 我 算是 娜娜 家风 荷园 开业 就 一直 在 这里 吃 每次 出去 回来 总想 ...


In [12]:
df = df[['text', 'sentiment']]

In [42]:
df[['text']].to_csv("output/dianping_cut.csv", index = False, header = False)

In [13]:
df.head()

,text,sentiment
0,口味 ： 不 知道 是 我口 高 了 ， 还是 这家 真 不怎么样 。 我 感觉 口味 ...,0
1,菜品 丰富 质量 好 ， 服务 也 不错 ！ 很 喜欢 ！,1
2,说真的 ， 不 晓得 有人 排队 的 理由 ， 香精 香精 香精 香精 ， 拜拜 ！,0
3,菜量 实惠 ， 上菜 还 算 比较 快 ， 疙瘩汤 喝出 了 秋日 的 暖意 ， 烧茄子 吃...,1
4,先说 我 算是 娜娜 家风 荷园 开业 就 一直 在 这里 吃 每次 出去 回来 总想 ...,1


In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

Using TensorFlow backend.


In [15]:
maxlen = 100
max_words = 10000

In [16]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df.text)
sequences = tokenizer.texts_to_sequences(df.text)

In [17]:
type(sequences)

list

In [18]:
len(sequences)
len(sequences[0])

150

In [19]:
len(sequences[:1][0])

150

In [20]:
[len(sequence) for sequence in sequences[:5]]

[150, 12, 16, 57, 253]

In [21]:
data = pad_sequences(sequences, maxlen=maxlen, value = 0.0)

长句子被剪裁(`maxlen`)，短句子被 0 padding

In [22]:
print(data.shape)
print(maxlen)

(2000, 100)
100


In [23]:
word_index = tokenizer.word_index

In [24]:
type(word_index)

dict

In [25]:
[str(key)+": "+str(value) for key, value in word_index.items()][:5]

['，: 1', '的: 2', '了: 3', '。: 4', '\xa0: 5']

In [26]:
labels = np.array(df.sentiment)

In [27]:
labels[:5]

array([0, 1, 0, 1, 1], dtype=int64)

In [28]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [29]:
training_samples = int(len(indices) * .8)
validation_samples = len(indices) - training_samples

In [30]:
training_samples

1600

In [31]:
validation_samples

400

In [32]:
X_train = data[:training_samples]
y_train = labels[:training_samples]
X_valid = data[training_samples: training_samples + validation_samples]
y_valid = labels[training_samples: training_samples + validation_samples]

In [33]:
X_train[:5,:5]

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [34]:
# !pip install gensim

In [35]:
import gensim

In [36]:
model = gensim.models.Word2Vec.load('refs/zh/zh.bin')

In [48]:
more_sentences = gensim.models.word2vec.Text8Corpus("output/dianping_cut.csv")  # 加载语料

```
ValueError: You must specify either total_examples or total_words, for proper job parameters updationand progress calculations. The usual value is total_examples=model.corpus_count.
```

In [67]:
model.build_vocab(more_sentences, update=True)  # 更新词汇表
model.train(more_sentences, total_examples=model.corpus_count, epochs=1)

(57687, 114396)

In [68]:
model.save("output/word2vec_online_training.model")